In [1]:
# Imports
import torch
import torch.nn as nn

In [4]:
inputs = torch.Tensor(1, 1, 28, 28)

In [5]:
# CNN layer1
conv1 = nn.Conv2d(1, 32, 3, padding=1)      # in_c=1, out_c=32
pool = nn.MaxPool2d(2)

# CNN layer2
conv2 = nn.Conv2d(32, 64, 3, padding=1)     # in_c=32, out_c=64

In [14]:
out=conv1(inputs)
out.shape           # batch_size=1, channel=32, size=28x28

torch.Size([1, 32, 28, 28])

In [15]:
out=pool(out)
out.shape           # batch_size=1, channel=32, size=14x14

torch.Size([1, 32, 14, 14])

In [16]:
out=conv2(out)
out.shape           # batch_size=1, channel=64, size=14x14

torch.Size([1, 64, 14, 14])

In [17]:
out=pool(out)
out.shape           # batch_size=1, channel=64, size=7x7

torch.Size([1, 64, 7, 7])

In [18]:
print(out.size(0))
print(out.size(1))
print(out.size(2))
print(out.size(3))

1
64
7
7


In [20]:
out = out.view(out.size(0), -1)
out.shape

torch.Size([1, 3136])

In [21]:
fc = nn.Linear(3136, 10)
out = fc(out)
out.shape

torch.Size([1, 10])

**CNN MNIST**

In [29]:
# Imports
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [24]:
device='cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device=='cuda':
    torch.cuda.manual_seed_all(777)

In [26]:
# parameters
learning_rate = 1e-3
training_epochs = 15
batch_size = 100

In [27]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)
mnist_test = dsets.MNIST(root='MNIST_data/',
                          train=False,
                          transform=transforms.ToTensor(),
                          download=True)

100.1%

Extracting MNIST.Data/MNIST\raw\train-images-idx3-ubyte.gz to MNIST.Data/MNIST\raw


0.0%

9.9%5%

Extracting MNIST.Data/MNIST\raw\train-labels-idx1-ubyte.gz to MNIST.Data/MNIST\raw


180.4%

Extracting MNIST.Data/MNIST\raw\t10k-images-idx3-ubyte.gz to MNIST.Data/MNIST\raw
Extracting MNIST.Data/MNIST\raw\t10k-labels-idx1-ubyte.gz to MNIST.Data/MNIST\raw
Processing...
Done!


c:\Users\dnjsg\anaconda3\envs\DL\lib\site-packages\torchvision\datasets\mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [28]:
# Data loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [43]:
class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        # Fully connected layer
        self.fc1 = nn.Linear(3*3*128, 625)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [44]:
# 모델 설정
model = CNN().to(device)
model

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=1152, out_features=625, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=625, out_features=10, bias=True)
)

In [45]:
# 생성된 모델 테스트
value = torch.Tensor(1, 1, 28, 28).to(device)
print( (model(value)).shape)

torch.Size([1, 10])


In [46]:
# cost / loss & optimizer
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [47]:
# Training
total_batch = len(data_loader)

for epoch in range(training_epochs):
    avg_cost=0
    
    for X, Y in data_loader:            # X 에는 images, Y 에는 labels
        X = X.to(device)  
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('Epoch {:6d}/{} Cost: {:.4f}'.format(
        epoch+1, training_epochs, avg_cost
    ))
print('learning finished')  

Epoch      1/15 Cost: 0.1664
Epoch      2/15 Cost: 0.0437
Epoch      3/15 Cost: 0.0291
Epoch      4/15 Cost: 0.0219
Epoch      5/15 Cost: 0.0166
Epoch      6/15 Cost: 0.0136
Epoch      7/15 Cost: 0.0114
Epoch      8/15 Cost: 0.0106
Epoch      9/15 Cost: 0.0100
Epoch     10/15 Cost: 0.0077
Epoch     11/15 Cost: 0.0056
Epoch     12/15 Cost: 0.0081
Epoch     13/15 Cost: 0.0053
Epoch     14/15 Cost: 0.0069
Epoch     15/15 Cost: 0.0037
learning finished


In [48]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy: ', accuracy.item())

c:\Users\dnjsg\anaconda3\envs\DL\lib\site-packages\torchvision\datasets\mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
c:\Users\dnjsg\anaconda3\envs\DL\lib\site-packages\torchvision\datasets\mnist.py:53: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy:  0.9768000245094299


**결론**

layer3 를 추가하지 않은 model 의 Accuracy 는 Accuracy:  0.9851999878883362 였지만,
 
layer3 를 추가한 model 의 Accuracy 는 Accuracy:  0.9768000245094299 로 더 높아졌다.

즉 layer 의 깊이와 cost 는 정비례하지는 않는다